In [1]:
import cv2
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  
import tensorflow as tf
from glob import glob
from yolov3.yolov4_config import Create_Yolo
from yolov3.utils_config import load_yolo_weights, detect_image, image_preprocess, postprocess_boxes_excel, nms, draw_bbox
from yolov3.configs_config import *
from PIL import Image

import os
import pandas as pd

if YOLO_TYPE == "yolov4":
    Darknet_weights = YOLO_V4_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V4_WEIGHTS
if YOLO_TYPE == "yolov3":
    Darknet_weights = YOLO_V3_TINY_WEIGHTS if TRAIN_YOLO_TINY else YOLO_V3_WEIGHTS

yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE)
load_yolo_weights(yolo, Darknet_weights) # use Darknet weights

2023-05-03 18:00:10.457463: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-05-03 18:00:10.457496: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: jh-desktop
2023-05-03 18:00:10.457500: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: jh-desktop
2023-05-03 18:00:10.457567: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.161.3
2023-05-03 18:00:10.457585: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.161.3
2023-05-03 18:00:10.457588: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.161.3
2023-05-03 18:00:10.457773: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

In [2]:
yolo = Create_Yolo(input_size=YOLO_INPUT_SIZE, CLASSES=TRAIN_CLASSES)
yolo.load_weights("./checkpoints/first/yolov3_custom") # use keras weights

In [3]:
def prt_loss(image_path ,yolo):
    input_size = 416
    iou_threshold = 0.45
    
    original_image      = cv2.imread(image_path)
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    original_image      = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
    image_data = image_preprocess(np.copy(original_image), [input_size, input_size])
    image_data = image_data[np.newaxis, ...].astype(np.float32)
    pred_bbox = yolo.predict(image_data)
    pred_bbox = [tf.reshape(x, (-1, tf.shape(x)[-1])) for x in pred_bbox]
    pred_bbox = tf.concat(pred_bbox, axis=0)
    bboxes, pred_coor, pred_conf  = postprocess_boxes_excel(pred_bbox, original_image, input_size, 0.0)
#     classes = np.argmax(pred_prob, axis=-1)
    scores = pred_conf 
    argmax_score = np.argmax(scores)
    max_score = np.max(scores)
    print_conf = pred_conf[argmax_score]
    
    return [print_conf]

In [4]:
def GIoU(bboxes_1, bboxes_2):
    # 1. calulate intersection over union
    area_1 = (bboxes_1[2] - bboxes_1[0]) * (bboxes_1[3] - bboxes_1[1])
    area_2 = (bboxes_2[2] - bboxes_2[0]) * (bboxes_2[3] - bboxes_2[1])
    
    intersection_wh = tf.minimum(bboxes_1[2:], bboxes_2[2:]) - tf.maximum(bboxes_1[ :2], bboxes_2[ :2])
    intersection_wh = tf.maximum(intersection_wh, 0)
    
    intersection = intersection_wh[0] * intersection_wh[1]
    union = (area_1 + area_2) - intersection
    
    ious = intersection / union
    
    # 2. (C - (A U B))/C
    C_wh = tf.maximum(bboxes_1[2:], bboxes_2[2:]) - tf.minimum(bboxes_1[:2], bboxes_2[:2])
    C_wh = C_wh
    C = C_wh[0] * C_wh[1]

    giou = ious - (C - union) / C
    return giou, ious

In [19]:
r1 = glob(r"teeth_including_lip_more/test/*.jpg")
r11 = glob(r"teeth_including_lip_more/train/*.jpg")
r5 = glob(r"/home/jh/Desktop/teeth/github_td/test2/*.jpg")
coco_anomaly =  glob(r"/home/jh/Desktop/teeth/github_td/val2017/*.jpg")
np.random.seed(42)
a = np.random.randint(5000, size =76)
a = np.sort(a)
print(a)
anomaly_76 = [coco_anomaly[i] for i in a]
anomaly_85 = glob(r"/home/jh/Desktop/teeth/github_td/ano/*.jpg")
brace_path = r"braces"
angled_path = r"angled"

[  34  130  161  189  241  337  466  474  600  646  769  775  860  878
  975  995 1016 1021 1076 1082 1184 1267 1275 1363 1478 1500 1528 1585
 1685 1806 1899 1955 1981 2041 2047 2061 2363 2391 2433 2435 2556 2558
 2612 2613 2734 2747 2824 2888 2904 2919 3005 3073 3092 3152 3171 3202
 3342 3385 3444 3461 3556 3772 3798 3843 3890 3943 4014 4117 4297 4426
 4551 4555 4658 4798 4843 4911]


In [20]:
train_names = []
valid_names = []
test_names = []
anomaly_names = []

In [21]:
coco_names = []
braces_names = []
angle_names = []

In [22]:
for i in range(len(r11)):
    train_names.append(os.path.basename(r11[i]))
for i in range(len(r1)):
    valid_names.append(os.path.basename(r1[i]))
for i in range(len(r5)):
    test_names.append(os.path.basename(r5[i]))
for i in range(len(anomaly_85)):
    anomaly_names.append(os.path.basename(anomaly_85[i]))

In [23]:
for i in range(len(coco_anomaly)):
    coco_names.append(os.path.basename(coco_anomaly[i]))
for i in range(len(os.listdir(brace_path))):
    braces_names.append(os.path.basename(os.listdir(brace_path)[i]))
for i in range(len(os.listdir(angled_path))):
    angle_names.append(os.path.basename(os.listdir(angled_path)[i]))

In [31]:
sorted(r1)

['teeth_including_lip_more/test/10146_2_3.jpg',
 'teeth_including_lip_more/test/10346_2_1.jpg',
 'teeth_including_lip_more/test/10355_2_1.jpg',
 'teeth_including_lip_more/test/144_2_2.jpg',
 'teeth_including_lip_more/test/174_2_1.jpg',
 'teeth_including_lip_more/test/19582_2_1.jpg',
 'teeth_including_lip_more/test/202_2_3.jpg',
 'teeth_including_lip_more/test/20451_2_1.jpg',
 'teeth_including_lip_more/test/20557_2_3.jpg',
 'teeth_including_lip_more/test/206_2_1.jpg',
 'teeth_including_lip_more/test/207_2_1.jpg',
 'teeth_including_lip_more/test/21511_2_1.jpg',
 'teeth_including_lip_more/test/21527_2_3.jpg',
 'teeth_including_lip_more/test/21770_2_1.jpg',
 'teeth_including_lip_more/test/23264_2_1.jpg',
 'teeth_including_lip_more/test/23456_2_1.jpg',
 'teeth_including_lip_more/test/23953_2_1.jpg',
 'teeth_including_lip_more/test/23956_2_3.jpg',
 'teeth_including_lip_more/test/24390_2_3.jpg',
 'teeth_including_lip_more/test/24391_2_3.jpg',
 'teeth_including_lip_more/test/24938_2_3.jpg',
 '

In [30]:
sorted(no_lip_valid)

['/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/10146_2_3.jpg 52,198,448,359,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/10346_2_1.jpg 101,186,420,299,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/10355_2_1.jpg 67,189,403,328,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/144_2_2.jpg 36,256,448,412,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/174_2_1.jpg 56,179,378,298,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/19582_2_1.jpg 107,219,365,368,0\n',
 '/home/jh/Desktop/conform_paper/test6/teeth_training_no_prob_no_lip_no_conf/teeth_excluding_lip_more/test/202_2_3.jpg 122,198,406,346,0\n',
 '/home/j

In [32]:
file = open("model_data/no_lip_more_train.txt", "r")
no_lip_train  = file.readlines()
no_lip_train = sorted(no_lip_train)
file.close()
file = open("model_data/no_lip_more_validation.txt", "r")
no_lip_valid = file.readlines()
no_lip_valid = sorted(no_lip_valid)
file.close()

In [33]:
GT_train = []
GT_valid = []

In [34]:
giou_ious_list_train = []
giou_ious_list_valid = []

In [35]:
#GT add
for i in range(len(r11)):
    bb = no_lip_train[i].split()[1].split(",")[0:4]
    bb = [int(item) for item in bb]
    GT_train.append(bb)
for i in range(len(r1)):
    bb = no_lip_valid[i].split()[1].split(",")[0:4]
    bb = [int(item) for item in bb]
    GT_valid.append(bb)

In [39]:
r11 = sorted(r11)
r1 = sorted(r1)

In [40]:
# train
for i in range(len(r11)):
    image, bboxes = detect_image(yolo, r11[i], "", input_size=YOLO_INPUT_SIZE, show=False, score_threshold=0.1, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    if len(bboxes) == 0 :
        giou_ious_list_train.append([0,0])
    else :
        giou,ious = GIoU(np.array(GT_train[i]), bboxes[0][:4])
        giou_ious_list_train.append([giou,ious])

In [41]:
# validation
for i in range(len(r1)):
    image, bboxes = detect_image(yolo, r1[i], "", input_size=YOLO_INPUT_SIZE,score_threshold=0.1, show=False, CLASSES=TRAIN_CLASSES, rectangle_colors=(255,0,0))
    if len(bboxes) == 0 :
        giou_ious_list_valid.append([0,0])
    else :
        giou,ious = GIoU(np.array(GT_valid[i]), bboxes[0][:4])
        giou_ious_list_valid.append([giou,ious])

In [45]:
np.mean(giou_ious_list_valid)

0.9036362168604536